# उदाहरण नमुना हॉटेल आणि फ्लाइट बुकिंग एजंट

हे समाधान तुम्हाला फ्लाइट तिकीट आणि हॉटेल बुक करण्यात मदत करेल. परिस्थिती अशी आहे: लंडन हीथ्रो LHR वरून न्यूयॉर्क JFK पर्यंत 20 फेब्रुवारी 2024 ला जाणे आणि 27 फेब्रुवारी 2025 ला परत येणे, ब्रिटिश एअरवेजने इकॉनॉमी क्लासमध्ये प्रवास करणे. न्यूयॉर्कमध्ये हिल्टन हॉटेलमध्ये राहायचे आहे, कृपया फ्लाइट आणि हॉटेलसाठी खर्च सांगा.


# Azure AI Agent Service सुरू करा आणि **.env** मधून कॉन्फिगरेशन माहिती मिळवा

### **.env**

एक .env फाइल तयार करा

**.env** मध्ये Azure AI Agent Service चा कनेक्शन स्ट्रिंग, AOAI वापरत असलेला मॉडेल, आणि संबंधित Google API Search सेवा API, ENDPOINT इत्यादींची माहिती असते.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "तुमच्या Azure AI Agent Service मॉडेल डिप्लॉयमेंटचे नाव"

[**NOTE**] तुम्हाला 100,000 रेट लिमिट (टोकन्स प्रति मिनिट) आणि 600 रेट लिमिट (रिक्वेस्ट प्रति मिनिट) असलेले मॉडेल लागेल.

  तुम्ही Azure AI Foundry - Model आणि Endpoint मध्ये मॉडेल मिळवू शकता.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "तुमच्या Azure AI Agent Service प्रोजेक्ट कनेक्शन स्ट्रिंग"

  तुम्ही AI Foundry Portal Screen मधील प्रोजेक्ट ओव्हरव्ह्यूमध्ये प्रोजेक्ट कनेक्शन स्ट्रिंग मिळवू शकता.

- **SERPAPI_SEARCH_API_KEY** = "तुमचा SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "तुमचा SERPAPI Search Endpoint"

Azure AI Agent Service चा Model Deployment Name आणि Project Connection String मिळवण्यासाठी तुम्हाला Azure AI Agent Service तयार करावी लागेल. [हा टेम्पलेट](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) वापरून ते थेट तयार करण्याची शिफारस केली जाते （***टीप:*** Azure AI Agent Service सध्या मर्यादित क्षेत्रात सेट केले आहे. [या लिंक](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) चा संदर्भ घेऊन क्षेत्र सेट करण्याची शिफारस केली जाते)

Agent ला SERPAPI वर प्रवेश करावा लागतो. [या लिंक](https://serpapi.com/searches) वापरून नोंदणी करण्याची शिफारस केली जाते. नोंदणी केल्यानंतर, तुम्ही एक अद्वितीय API KEY आणि ENDPOINT मिळवू शकता.


# सेटअप

हे नोटबुक चालवण्यासाठी, तुम्हाला खात्री करावी लागेल की आवश्यक लायब्ररी `pip install -r requirements.txt` चालवून स्थापित केल्या आहेत.


In [ ]:
from semantic_kernel import __version__

__version__

तुमचा सेमॅंटिक कर्नल आवृत्ती कमीत कमी 1.27.2 असावी.


तुमची .env फाइल सेटिंग आणि संसाधने लोड करा कृपया सुनिश्चित करा की तुम्ही तुमच्या कीज आणि सेटिंग जोडल्या आहेत आणि स्थानिक .env फाइल तयार केली आहे.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Azure मध्ये लॉग इन करा

आता तुम्हाला Azure मध्ये लॉग इन करणे आवश्यक आहे. टर्मिनल उघडा आणि खालील कमांड चालवा:

```bash
az login
```

ही कमांड तुम्हाला तुमची Azure क्रेडेन्शियल्स प्रविष्ट करण्यास सांगेल, ज्यामुळे Azure AI Agent सेवा योग्यरित्या कार्य करू शकेल.


# स्पष्टीकरण:
ही एक व्हेरिएबल आहे जी SERP (सर्च इंजिन रिझल्ट्स पेज) API सेवा वापरण्यासाठी API की साठवते. API की ही एक अद्वितीय ओळख आहे जी तुमच्या खात्याशी संबंधित विनंत्यांची प्रमाणीकरण करण्यासाठी वापरली जाते.

उद्देश: या ओळीचा उद्देश API की एका व्हेरिएबलमध्ये साठवणे आहे, जेणेकरून ती SERP API सेवेसाठी विनंत्यांचे प्रमाणीकरण करण्यासाठी वापरता येईल. सेवा वापरण्यासाठी आणि शोध करण्यासाठी API की आवश्यक आहे.

SERP API की कशी मिळवायची: SERP API की मिळवण्यासाठी, https://serpapi.com वर खालील सामान्य चरणांचे अनुसरण करा (तुम्ही वापरत असलेल्या विशिष्ट SERP API सेवेनुसार अचूक चरण वेगळे असू शकतात):

SERP API सेवा निवडा: अनेक SERP API सेवा उपलब्ध आहेत, जसे की SerpAPI, Google Custom Search JSON API, आणि इतर. तुमच्या गरजेनुसार योग्य सेवा निवडा.

खात्यासाठी साइन अप करा: निवडलेल्या SERP API सेवेसाठी वेबसाइटवर जा आणि खात्यासाठी साइन अप करा. तुम्हाला काही मूलभूत माहिती द्यावी लागेल आणि तुमचा ईमेल पत्ता सत्यापित करावा लागेल.

API की तयार करा: साइन अप केल्यानंतर, तुमच्या खात्यात लॉग इन करा आणि API विभाग किंवा डॅशबोर्डवर जा. नवीन API की तयार करण्याचा किंवा निर्माण करण्याचा पर्याय शोधा.
API की तुमच्या .env फाइलमध्ये कॉपी करा.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# स्पष्टीकरण:
BASE_URL: ही एक व्हेरिएबल आहे जी SERP API एन्डपॉइंटसाठी बेस URL साठवते. BASE_URL हे नाव अशा प्रकारच्या URL साठी वापरले जाते जे API विनंत्या करण्यासाठी सुरुवातीचा बिंदू दर्शवते.
'https://serpapi.com/search':

ही BASE_URL व्हेरिएबलला दिलेली वास्तविक URL स्ट्रिंग आहे. ही SERP API वापरून शोध क्वेरी करण्यासाठी एन्डपॉइंट दर्शवते.

# उद्दिष्ट:
या ओळीचा उद्देश म्हणजे SERP API साठी बेस URL साठवणारी एक स्थिरता (constant) परिभाषित करणे. ही URL शोध कार्ये करण्यासाठी API विनंत्या तयार करण्यासाठी सुरुवातीचा बिंदू म्हणून वापरली जाईल.

# उपयोग:
बेस URL एका व्हेरिएबलमध्ये परिभाषित केल्याने, जेव्हा जेव्हा तुम्हाला SERP API ला विनंती करायची असेल तेव्हा तुम्ही ती सहजपणे कोडमध्ये पुन्हा वापरू शकता. यामुळे तुमचा कोड अधिक देखभालयोग्य होतो आणि अनेक ठिकाणी URL हार्डकोड केल्यामुळे होणाऱ्या चुका कमी होतात. सध्याचा उदाहरण म्हणजे https://serpapi.com/search?engine=bing जो Bing शोध API वापरत आहे. वेगवेगळ्या API साठी तुम्ही https://Serpapi.com येथे निवड करू शकता.


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# स्पष्टीकरण:

हे तुमच्या प्लगइन कोडचे स्थान आहे.

क्लास डिफिनिशन: `class BookingPlugin`: BookingPlugin नावाचा एक वर्ग परिभाषित करतो, ज्यामध्ये हॉटेल आणि फ्लाइट बुकिंगसाठी पद्धती (methods) आहेत.

हॉटेल बुकिंग पद्धत:

- `@kernel_function(description="booking hotel")`: हॉटेल बुकिंगसाठी ही पद्धत एक kernel function म्हणून वर्णन करणारा डेकोरेटर.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Annotated parameters आणि return type सह हॉटेल बुकिंगसाठी पद्धत परिभाषित करते.

ही पद्धत हॉटेल बुकिंग विनंतीसाठी parameters चा एक डिक्शनरी तयार करते आणि SERP API ला GET विनंती पाठवते. ती प्रतिसाद स्थिती तपासते आणि यशस्वी झाल्यास हॉटेलची माहिती परत करते, अन्यथा विनंती अयशस्वी झाल्यास None परत करते.

फ्लाइट बुकिंग पद्धत:

- `@kernel_function(description="booking flight")`: फ्लाइट बुकिंगसाठी ही पद्धत एक kernel function म्हणून वर्णन करणारा डेकोरेटर.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Annotated parameters आणि return type सह फ्लाइट बुकिंगसाठी पद्धत परिभाषित करते.

ही पद्धत आउटबाउंड आणि रिटर्न फ्लाइट विनंत्यांसाठी parameters चे डिक्शनरी तयार करते आणि SERP API ला GET विनंती पाठवते. ती प्रतिसाद स्थिती तपासते आणि यशस्वी झाल्यास फ्लाइटची माहिती result string मध्ये जोडते, अन्यथा त्रुटी संदेश प्रिंट करते. ही पद्धत फ्लाइट माहिती असलेला result string परत करते.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# स्पष्टीकरण:
आयात स्टेटमेंट्स: Azure क्रेडेन्शियल्स, AI एजंट, चॅट मेसेज कंटेंट, लेखकाची भूमिका, आणि कर्नल फंक्शन डेकोरेटरसाठी आवश्यक मॉड्यूल्स आयात करा.

असिंक्रोनस कॉन्टेक्स्ट मॅनेजर: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): हे असिंक्रोनस कॉन्टेक्स्ट मॅनेजर सेट करते जे Azure क्रेडेन्शियल्स हाताळते आणि AI एजंट क्लायंट तयार करते.

एजंटचे नाव आणि सूचना:
- `AGENT_NAME = "BookingAgent"`: एजंटचे नाव परिभाषित करते.
- `AGENT_INSTRUCTIONS = """..."""`: एजंटला बुकिंग विनंत्या कशा हाताळायच्या याबद्दल तपशीलवार सूचना प्रदान करते.

एजंट डिफिनिशन तयार करा: `agent_definition = await client.agents.create_agent(...)`: दिलेल्या मॉडेल, नाव, आणि सूचनांसह एजंट डिफिनिशन तयार करते.

AzureAI एजंट तयार करा: `agent = AzureAIAgent(...)`: क्लायंट, एजंट डिफिनिशन, आणि परिभाषित प्लगइन वापरून AzureAI एजंट तयार करते.

थ्रेड तयार करा: `thread: AzureAIAgentThread | None = None`: एजंटसाठी थ्रेड तयार करा. थ्रेड प्रथम तयार करणे आवश्यक नाही - जर `None` मूल्य दिले गेले असेल, तर पहिल्या कॉल दरम्यान नवीन थ्रेड तयार केला जाईल आणि प्रतिसादाचा भाग म्हणून परत केला जाईल.

वापरकर्ता इनपुट्स: `user_inputs = ["..."]`: एजंट प्रक्रियेसाठी वापरकर्त्याच्या इनपुट्सची यादी परिभाषित करते.

शेवटी ब्लॉकमध्ये, संसाधने साफ करण्यासाठी थ्रेड आणि एजंट हटवा.


# प्रमाणीकरण

`DefaultAzureCredential` वर्ग हा Azure SDK for Python चा एक भाग आहे. तो Azure सेवांसोबत प्रमाणीकरण करण्यासाठी एक डिफॉल्ट पद्धत प्रदान करतो. तो विशिष्ट क्रमाने अनेक पद्धतींचा वापर करून प्रमाणीकरण करण्याचा प्रयत्न करतो, जसे की पर्यावरणीय चल (environment variables), व्यवस्थापित ओळख (managed identity), आणि Azure CLI क्रेडेन्शियल्स.

असमकालिक ऑपरेशन्स: `aio` मॉड्यूल सूचित करतो की `DefaultAzureCredential` वर्ग असमकालिक ऑपरेशन्सला समर्थन देतो. याचा अर्थ असा की तुम्ही `asyncio` चा वापर करून नॉन-ब्लॉकिंग प्रमाणीकरण विनंत्या करू शकता.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
